# 2018년 코트라 데이터 모델링

저희는 2018년 데이터 분석 및 모델링을 통해 가장 낮은 `RMSE`를 찾으며 분석과 모델링을 바탕으로 2019년 수입금액을 예측하였습니다. <br>

머신러닝 모델을 훈련하는 과정을 알아보면, 크게 단일 모델과 앙상블 모델로 나눌 수 습니다. <br>
 <br>`단일 모델`은 하나의 모델만 사용해 최종값을 도출하는 모델을 의미합니다. <br>

`앙상블 모델`은 여러 개의 모델을 적절하게 결합해 최종값을 도출하는 의미합니다. <br>

 <br>이번에는 주로 단일 모델의 조합으로 결과값을 내는 앙상블 모델을 사용해 일반화 `앙상블 모델` 학습법에는 `배깅(Bagging)과 부스팅(Boosting)`방식을 진행했습니다. <br>

 <br>`배깅(bagging)`과 `부스팅(boosting)`을 통해 변수를 선정하며 `A/B 테스트`를 통해 최적의 변수를 선정하였습니다.  
모델의 일반화 성능을 높이기 위해 `랜덤 포레스트(RandomForest)`를 주 모델로 선정하였습니다. 



## 2018년 모델링과 정답 데이터를 가지고 2019년을 예측 값 도출




## INDEX


### 1. 모델 구축을 위한 모듈 및 데이터 로드


### 2 이상치 제거 후 train으로 학습 / test는 전체 데이터 예측
#### 2.1 피처 스케일링
#### 2.2 Submission csv 만들기



### 3. 2018가장 좋은 성능 모델링으로 2019년 정답 예측



### 4. 결론 
#### 2019 예측 submission도출


# 1.모델 구축을 위한 모듈 및 데이터 로드

In [81]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#내부에 결과를 출력하도록 설정

%matplotlib inline 

#시각화 한글 폰트 설정
#맥 사용자
plt.rc('font', family ='AppleGothic')

#마이너스 기호 출력
plt.rc('axes', unicode_minus=False)

#분석에 문제가 없는 경고 메세지는 숨긴다.
import warnings 
warnings.filterwarnings('ignore')

# 2. 이상치 제거 후 test으로 학습 / test6는 전체 데이터 예측

In [82]:
test_all = pd.read_csv('공모전데이터_예측용_KOTRA_0525.csv')

In [83]:
test6 = pd.read_csv('test6.csv')

In [84]:
test6

,UNC_YEAR,HSCD,COUNTRYCD,COUNTRYNM,TRADE_COUNTRYCD,TRADE_HSCD,TARIFF_AVG,SNDIST,NY_GDP_MKTP_CD,NY_GDP_MKTP_CD_1Y,...,2018_lpi,2017_fl,2018_fl,18fl_diff,TARIFF_AVG_sum,TRADE_HSCD_COUNTRYCD_sum,TARIFF_AVG_mean,TRADE_HSCD_COUNTRYCD_mean,TRADE_HSCD_COUNTRYCD_ratio,KR_TRADE_HSCD_COUNTRYCD_y
0,2018,1,12,Algeria,42196119729,2.223464e+10,27.000000,3735.047389,175405660377,170163165961,...,2.450000,24.803750,28.280480,0.140170,377.3,1.766479e+10,8.983333,4.108091e+08,0.000384,7657
1,2018,1,36,Australia,240422685574,2.223464e+10,1.200000,11947.511360,1432881172002,1329188475752,...,3.750000,1982.586171,2027.637110,0.022723,377.3,1.766479e+10,8.983333,4.108091e+08,0.027318,5294788
2,2018,1,40,Austria,186965232670,2.223464e+10,2.800000,4005.020029,455094861902,417237869116,...,4.030000,391.268000,373.511000,-0.045383,377.3,1.766479e+10,8.983333,4.108091e+08,0.028301,335818
3,2018,1,56,Belgium,461444842911,2.223464e+10,2.800000,4501.782826,543734366831,502698069367,...,4.040000,1574.378609,1285.344187,-0.183586,377.3,1.766479e+10,8.983333,4.108091e+08,0.043539,173709
4,2018,1,76,Brazil,185290138433,2.223464e+10,17.100000,10015.769070,1885482534238,2062831045936,...,2.990000,1736.549227,1845.647760,0.062825,377.3,1.766479e+10,8.983333,4.108091e+08,0.002311,122850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21184,2018,1,784,United Arab Emirates,254510973245,2.092602e+10,4.389127,5404.902374,422215043585,385605506855,...,3.960000,16616.244610,15962.904430,-0.039319,0.0,1.835296e+10,4.389563,4.268130e+08,0.010695,881214
21185,2018,1,792,Turkey,234394880098,2.092602e+10,4.389127,3810.711560,778381859841,858988610574,...,3.150000,4800.237745,5949.212206,0.239358,0.0,1.835296e+10,4.389563,4.268130e+08,0.006081,9545535
21186,2018,1,818,Egypt,94326188483,2.092602e+10,4.389127,4064.977456,249712999437,235733695652,...,3.361892,3224.890471,3338.708215,0.035294,0.0,1.835296e+10,4.389563,4.268130e+08,0.001872,1313545
21187,2018,1,826,United Kingdom,723877103929,2.092602e+10,4.389127,4402.897106,2860667727552,2666229179958,...,3.990000,5915.641000,6198.370000,0.047793,0.0,1.835296e+10,4.389563,4.268130e+08,0.067495,19876811


In [85]:
#학습 데이터 복사본 생성
test = test6.copy()

In [86]:
#TRADE_HSCD_COUNTRYCD 이상치를 탐색을 위한 수치 정의
Q1 = test['TRADE_HSCD_COUNTRYCD'].quantile(0.25)
Q3 = test['TRADE_HSCD_COUNTRYCD'].quantile(0.75)
IQR = Q3-Q1


#실제 TRADE_HSCD_COUNTRYCD 이상치 탐색
df_outlier= test.loc[(test['TRADE_HSCD_COUNTRYCD'] <(Q1-1.5*IQR))|
             (test['TRADE_HSCD_COUNTRYCD']>(Q3+1.5*IQR))].sort_values(
by=['COUNTRYNM'],axis=0, ascending=False)[['COUNTRYNM','HSCD','TRADE_HSCD_COUNTRYCD']]




In [87]:
#이상치를 제거한 데이터만 선택
test = test.loc[(test['TRADE_HSCD_COUNTRYCD'] >(Q1-1.5*IQR)) &
             (test['TRADE_HSCD_COUNTRYCD']<(Q3+1.5*IQR))]

In [88]:
#학습할 데이터
test

,UNC_YEAR,HSCD,COUNTRYCD,COUNTRYNM,TRADE_COUNTRYCD,TRADE_HSCD,TARIFF_AVG,SNDIST,NY_GDP_MKTP_CD,NY_GDP_MKTP_CD_1Y,...,2018_lpi,2017_fl,2018_fl,18fl_diff,TARIFF_AVG_sum,TRADE_HSCD_COUNTRYCD_sum,TARIFF_AVG_mean,TRADE_HSCD_COUNTRYCD_mean,TRADE_HSCD_COUNTRYCD_ratio,KR_TRADE_HSCD_COUNTRYCD_y
0,2018,1,12,Algeria,42196119729,2.223464e+10,27.000000,3735.047389,175405660377,170163165961,...,2.450000,24.803750,28.280480,0.140170,377.3,1.766479e+10,8.983333,4.108091e+08,0.000384,7657
4,2018,1,76,Brazil,185290138433,2.223464e+10,17.100000,10015.769070,1885482534238,2062831045936,...,2.990000,1736.549227,1845.647760,0.062825,377.3,1.766479e+10,8.983333,4.108091e+08,0.002311,122850
5,2018,1,104,Myanmar,21113013351,2.223464e+10,13.400000,5947.065298,76168043982,68945867478,...,2.300000,4.833936,4.742166,-0.018985,377.3,1.766479e+10,8.983333,4.108091e+08,0.000854,631082
7,2018,1,144,Sri Lanka,18419136335,2.223464e+10,30.000000,6910.058402,88425889592,87428125558,...,2.600000,398.473464,436.198000,0.094673,377.3,1.766479e+10,8.983333,4.108091e+08,0.000140,44837
8,2018,1,152,Chile,75903274076,2.223464e+10,3.300000,9682.793339,298258019275,277044950259,...,3.320000,1237.960141,1226.441990,-0.009304,377.3,1.766479e+10,8.983333,4.108091e+08,0.003060,63715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21181,2018,1,710,South Africa,96117430662,2.092602e+10,4.389127,7913.087574,368288939768,349554116684,...,3.380000,833.347947,696.730006,-0.163939,0.0,1.835296e+10,4.389563,4.268130e+08,0.001233,377880
21183,2018,1,757,Switzerland,289593607423,2.092602e+10,4.389127,4269.987897,705140620047,679950481623,...,3.900000,1581.352360,1841.314000,0.164392,0.0,1.835296e+10,4.389563,4.268130e+08,0.012262,2555079
21184,2018,1,784,United Arab Emirates,254510973245,2.092602e+10,4.389127,5404.902374,422215043585,385605506855,...,3.960000,16616.244610,15962.904430,-0.039319,0.0,1.835296e+10,4.389563,4.268130e+08,0.010695,881214
21185,2018,1,792,Turkey,234394880098,2.092602e+10,4.389127,3810.711560,778381859841,858988610574,...,3.150000,4800.237745,5949.212206,0.239358,0.0,1.835296e+10,4.389563,4.268130e+08,0.006081,9545535


In [89]:
#테스트할 데이터
test6

,UNC_YEAR,HSCD,COUNTRYCD,COUNTRYNM,TRADE_COUNTRYCD,TRADE_HSCD,TARIFF_AVG,SNDIST,NY_GDP_MKTP_CD,NY_GDP_MKTP_CD_1Y,...,2018_lpi,2017_fl,2018_fl,18fl_diff,TARIFF_AVG_sum,TRADE_HSCD_COUNTRYCD_sum,TARIFF_AVG_mean,TRADE_HSCD_COUNTRYCD_mean,TRADE_HSCD_COUNTRYCD_ratio,KR_TRADE_HSCD_COUNTRYCD_y
0,2018,1,12,Algeria,42196119729,2.223464e+10,27.000000,3735.047389,175405660377,170163165961,...,2.450000,24.803750,28.280480,0.140170,377.3,1.766479e+10,8.983333,4.108091e+08,0.000384,7657
1,2018,1,36,Australia,240422685574,2.223464e+10,1.200000,11947.511360,1432881172002,1329188475752,...,3.750000,1982.586171,2027.637110,0.022723,377.3,1.766479e+10,8.983333,4.108091e+08,0.027318,5294788
2,2018,1,40,Austria,186965232670,2.223464e+10,2.800000,4005.020029,455094861902,417237869116,...,4.030000,391.268000,373.511000,-0.045383,377.3,1.766479e+10,8.983333,4.108091e+08,0.028301,335818
3,2018,1,56,Belgium,461444842911,2.223464e+10,2.800000,4501.782826,543734366831,502698069367,...,4.040000,1574.378609,1285.344187,-0.183586,377.3,1.766479e+10,8.983333,4.108091e+08,0.043539,173709
4,2018,1,76,Brazil,185290138433,2.223464e+10,17.100000,10015.769070,1885482534238,2062831045936,...,2.990000,1736.549227,1845.647760,0.062825,377.3,1.766479e+10,8.983333,4.108091e+08,0.002311,122850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21184,2018,1,784,United Arab Emirates,254510973245,2.092602e+10,4.389127,5404.902374,422215043585,385605506855,...,3.960000,16616.244610,15962.904430,-0.039319,0.0,1.835296e+10,4.389563,4.268130e+08,0.010695,881214
21185,2018,1,792,Turkey,234394880098,2.092602e+10,4.389127,3810.711560,778381859841,858988610574,...,3.150000,4800.237745,5949.212206,0.239358,0.0,1.835296e+10,4.389563,4.268130e+08,0.006081,9545535
21186,2018,1,818,Egypt,94326188483,2.092602e+10,4.389127,4064.977456,249712999437,235733695652,...,3.361892,3224.890471,3338.708215,0.035294,0.0,1.835296e+10,4.389563,4.268130e+08,0.001872,1313545
21187,2018,1,826,United Kingdom,723877103929,2.092602e+10,4.389127,4402.897106,2860667727552,2666229179958,...,3.990000,5915.641000,6198.370000,0.047793,0.0,1.835296e+10,4.389563,4.268130e+08,0.067495,19876811


#### 학습할 데이터 정리

In [90]:
#문자 형태의 변수 추출
object_column = list(test.columns[test.dtypes=='object'])
#문자형 변수 제거 
test = test.drop(object_column, axis='columns')

In [91]:
test.drop(['UNC_YEAR'],axis=1, inplace=True)

In [92]:
test.drop(['COUNTRYCD'],axis=1, inplace=True)

In [93]:
test.drop(['18LCL_diff'],axis=1, inplace=True)

In [94]:
test

,HSCD,TRADE_COUNTRYCD,TRADE_HSCD,TARIFF_AVG,SNDIST,NY_GDP_MKTP_CD,NY_GDP_MKTP_CD_1Y,SP_POP_TOTL,PA_NUS_FCRF,IC_BUS_EASE_DFRN_DB,...,2018_lpi,2017_fl,2018_fl,18fl_diff,TARIFF_AVG_sum,TRADE_HSCD_COUNTRYCD_sum,TARIFF_AVG_mean,TRADE_HSCD_COUNTRYCD_mean,TRADE_HSCD_COUNTRYCD_ratio,KR_TRADE_HSCD_COUNTRYCD_y
0,1,42196119729,2.223464e+10,27.000000,3735.047389,175405660377,170163165961,42228429.0,100.691433,46.22686,...,2.450000,24.803750,28.280480,0.140170,377.3,1.766479e+10,8.983333,4.108091e+08,0.000384,7657
4,1,185290138433,2.223464e+10,17.100000,10015.769070,1885482534238,2062831045936,209469333.0,2.156089,55.80145,...,2.990000,1736.549227,1845.647760,0.062825,377.3,1.766479e+10,8.983333,4.108091e+08,0.002311,122850
5,1,21113013351,2.223464e+10,13.400000,5947.065298,76168043982,68945867478,53708395.0,933.570454,42.97482,...,2.300000,4.833936,4.742166,-0.018985,377.3,1.766479e+10,8.983333,4.108091e+08,0.000854,631082
7,1,18419136335,2.223464e+10,30.000000,6910.058402,88425889592,87428125558,21670000.0,129.069031,59.95122,...,2.600000,398.473464,436.198000,0.094673,377.3,1.766479e+10,8.983333,4.108091e+08,0.000140,44837
8,1,75903274076,2.223464e+10,3.300000,9682.793339,298258019275,277044950259,18729160.0,495.272878,71.95100,...,3.320000,1237.960141,1226.441990,-0.009304,377.3,1.766479e+10,8.983333,4.108091e+08,0.003060,63715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21181,1,96117430662,2.092602e+10,4.389127,7913.087574,368288939768,349554116684,57779622.0,9.655056,65.32778,...,3.380000,833.347947,696.730006,-0.163939,0.0,1.835296e+10,4.389563,4.268130e+08,0.001233,377880
21183,1,289593607423,2.092602e+10,4.389127,4269.987897,705140620047,679950481623,8514329.0,0.926904,76.60784,...,3.900000,1581.352360,1841.314000,0.164392,0.0,1.835296e+10,4.389563,4.268130e+08,0.012262,2555079
21184,1,254510973245,2.092602e+10,4.389127,5404.902374,422215043585,385605506855,9630959.0,3.672500,79.30036,...,3.960000,16616.244610,15962.904430,-0.039319,0.0,1.835296e+10,4.389563,4.268130e+08,0.010695,881214
21185,1,234394880098,2.092602e+10,4.389127,3810.711560,778381859841,858988610574,82319724.0,1.903768,70.91450,...,3.150000,4800.237745,5949.212206,0.239358,0.0,1.835296e+10,4.389563,4.268130e+08,0.006081,9545535


In [95]:
#평균 값으로 대치하기
test = test.where(pd.notnull(test), test.mean(), axis='columns')

#### 테스트할 데이터 정리

In [96]:
#문자 형태의 변수 추출
object_column = list(test.columns[test.dtypes=='object'])
#문자형 변수 제거 
test6 = test6.drop(object_column, axis='columns')

In [97]:
test6.drop(['UNC_YEAR'],axis=1, inplace=True)

In [98]:
test6.drop(['COUNTRYCD'],axis=1, inplace=True)

In [99]:
test6.drop(['18LCL_diff'],axis=1, inplace=True)

In [100]:
test6

,HSCD,COUNTRYNM,TRADE_COUNTRYCD,TRADE_HSCD,TARIFF_AVG,SNDIST,NY_GDP_MKTP_CD,NY_GDP_MKTP_CD_1Y,SP_POP_TOTL,PA_NUS_FCRF,...,2018_lpi,2017_fl,2018_fl,18fl_diff,TARIFF_AVG_sum,TRADE_HSCD_COUNTRYCD_sum,TARIFF_AVG_mean,TRADE_HSCD_COUNTRYCD_mean,TRADE_HSCD_COUNTRYCD_ratio,KR_TRADE_HSCD_COUNTRYCD_y
0,1,Algeria,42196119729,2.223464e+10,27.000000,3735.047389,175405660377,170163165961,42228429.0,100.691433,...,2.450000,24.803750,28.280480,0.140170,377.3,1.766479e+10,8.983333,4.108091e+08,0.000384,7657
1,1,Australia,240422685574,2.223464e+10,1.200000,11947.511360,1432881172002,1329188475752,24982688.0,1.035843,...,3.750000,1982.586171,2027.637110,0.022723,377.3,1.766479e+10,8.983333,4.108091e+08,0.027318,5294788
2,1,Austria,186965232670,2.223464e+10,2.800000,4005.020029,455094861902,417237869116,8840521.0,1.139000,...,4.030000,391.268000,373.511000,-0.045383,377.3,1.766479e+10,8.983333,4.108091e+08,0.028301,335818
3,1,Belgium,461444842911,2.223464e+10,2.800000,4501.782826,543734366831,502698069367,11427054.0,1.139000,...,4.040000,1574.378609,1285.344187,-0.183586,377.3,1.766479e+10,8.983333,4.108091e+08,0.043539,173709
4,1,Brazil,185290138433,2.223464e+10,17.100000,10015.769070,1885482534238,2062831045936,209469333.0,2.156089,...,2.990000,1736.549227,1845.647760,0.062825,377.3,1.766479e+10,8.983333,4.108091e+08,0.002311,122850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21184,1,United Arab Emirates,254510973245,2.092602e+10,4.389127,5404.902374,422215043585,385605506855,9630959.0,3.672500,...,3.960000,16616.244610,15962.904430,-0.039319,0.0,1.835296e+10,4.389563,4.268130e+08,0.010695,881214
21185,1,Turkey,234394880098,2.092602e+10,4.389127,3810.711560,778381859841,858988610574,82319724.0,1.903768,...,3.150000,4800.237745,5949.212206,0.239358,0.0,1.835296e+10,4.389563,4.268130e+08,0.006081,9545535
21186,1,Egypt,94326188483,2.092602e+10,4.389127,4064.977456,249712999437,235733695652,98423595.0,6.870325,...,3.361892,3224.890471,3338.708215,0.035294,0.0,1.835296e+10,4.389563,4.268130e+08,0.001872,1313545
21187,1,United Kingdom,723877103929,2.092602e+10,4.389127,4402.897106,2860667727552,2666229179958,66460344.0,0.639661,...,3.990000,5915.641000,6198.370000,0.047793,0.0,1.835296e+10,4.389563,4.268130e+08,0.067495,19876811


## 2.1 피처 스케일링

In [101]:
#스케일링 작업 같은 범위로 압축 
# 피처 각각에 대한 scaling을 수행하는 함수를 정의합니다.
def standard_scaling(df, scale_columns):
    for col in scale_columns:
        series_mean = df[col].mean()
        series_std = df[col].std()
        df[col] = df[col].apply(lambda x: (x-series_mean)/series_std)
    return df

#컨셉은 '각각의 평균을 표준편차로 나눈 값은 수렴한다'라고 생각됩니다.
#확률과 통계에서 자주 나오는 표준편차와 분산은 보통 -1~+1 사이에서 분포한다는 점과 연관하여 생각해보았습니다.

- 스케일링 필요 없는 변수


- gdp_diff
- country_depend
- 18FCL_diff                                
- 18EMPTY_diff                
- 18Total_diff
- 18hang_diff
- 18fl_diff
- TRADE_HSCD_COUNTRYCD_ratio
- 'FTA',
- 'OECD',
- 'WTO',
- 'ASEM',
- 'ASEAN',
- 'FEALAC',
- 'HSCD'
- '2019_KR'
- KR_TRADE_HSCD_COUNTRYCD'
- KR_TRADE_HSCD_COUNTRYCD_19

In [102]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18464 entries, 0 to 21186
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   HSCD                        18464 non-null  int64  
 1   TRADE_COUNTRYCD             18464 non-null  int64  
 2   TRADE_HSCD                  18464 non-null  float64
 3   TARIFF_AVG                  18464 non-null  float64
 4   SNDIST                      18464 non-null  float64
 5   NY_GDP_MKTP_CD              18464 non-null  int64  
 6   NY_GDP_MKTP_CD_1Y           18464 non-null  int64  
 7   SP_POP_TOTL                 18464 non-null  float64
 8   PA_NUS_FCRF                 18464 non-null  float64
 9   IC_BUS_EASE_DFRN_DB         18464 non-null  float64
 10  KMDIST                      18464 non-null  float64
 11  TRADE_HSCD_COUNTRYCD        18464 non-null  float64
 12  gdp_diff                    18464 non-null  float64
 13  1_gdp                       184

In [103]:
# 피처 각각에 대한 scaling을 수행합니다.
scale_columns = [
    #'HSCD',
 'TRADE_COUNTRYCD',
 'TRADE_HSCD',
 'TARIFF_AVG',
 'SNDIST',
 'NY_GDP_MKTP_CD',
 'NY_GDP_MKTP_CD_1Y',
 'SP_POP_TOTL',
 'PA_NUS_FCRF',
 'IC_BUS_EASE_DFRN_DB',
 'KMDIST',
 #'gdp_diff',
 '1_gdp',
 'item_depend',
 #'country_depend',
 'item_md_rv',
 'sndist_mean',
 'TRADE_HSCD_COUNTRYCD',
 #'FTA',
 #'OECD',
 #'WTO',
 #'ASEM',
 #'ASEAN',
 #'FEALAC',
 '2018_FCL',
 '2018_LCL',
 '2018_EMPTY',
 '2018_Total',
 '2019_FCL',
 '2019_LCL',
 '2019_EMPTY',
 '2019_Total',
 #'18FCL_diff',
 #'18EMPTY_diff',
 #'18Total_diff',
 '2017_hang',
 '2018_hang',
 #'18hang_diff',
 '2018_lpi',
 '2017_fl',
 '2018_fl',
 #'18fl_diff',
 'TARIFF_AVG_sum',
 'TRADE_HSCD_COUNTRYCD_mean',
 'TARIFF_AVG_mean',
 'TRADE_HSCD_COUNTRYCD_sum',  
 #'TRADE_HSCD_COUNTRYCD_ratio'
    
]
picher_df = standard_scaling(test, scale_columns)
picher_df



,HSCD,TRADE_COUNTRYCD,TRADE_HSCD,TARIFF_AVG,SNDIST,NY_GDP_MKTP_CD,NY_GDP_MKTP_CD_1Y,SP_POP_TOTL,PA_NUS_FCRF,IC_BUS_EASE_DFRN_DB,...,2018_lpi,2017_fl,2018_fl,18fl_diff,TARIFF_AVG_sum,TRADE_HSCD_COUNTRYCD_sum,TARIFF_AVG_mean,TRADE_HSCD_COUNTRYCD_mean,TRADE_HSCD_COUNTRYCD_ratio,KR_TRADE_HSCD_COUNTRYCD_y
0,1,-0.657357,0.452330,2.603955,-1.027045,-0.413188,-0.418648,-0.251327,-0.271304,-2.319520,...,-1.676935,-0.706368,-0.710815,0.140170,2.048616,0.399641,1.982026,0.363821,0.000384,7657
4,1,-0.298882,0.452330,1.452287,1.362280,0.243263,0.367198,0.377234,-0.293355,-1.409419,...,-0.649483,-0.293019,-0.284161,0.062825,2.048616,0.399641,1.982026,0.363821,0.002311,122850
5,1,-0.710173,0.452330,1.021866,-0.185544,-0.451282,-0.460674,-0.208181,-0.084915,-2.628639,...,-1.962338,-0.711191,-0.716341,-0.018985,2.048616,0.399641,1.982026,0.363821,0.000854,631082
7,1,-0.716922,0.452330,2.952946,0.180799,-0.446577,-0.453000,-0.328594,-0.264954,-1.014968,...,-1.391531,-0.616135,-0.615051,0.094673,2.048616,0.399641,1.982026,0.363821,0.000140,44837
8,1,-0.572914,0.452330,-0.153068,1.235608,-0.366028,-0.374270,-0.339647,-0.183001,0.125657,...,-0.021595,-0.413418,-0.429529,-0.009304,2.048616,0.399641,1.982026,0.363821,0.003060,63715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21181,1,-0.522275,0.401981,-0.026370,0.562374,-0.339145,-0.344164,-0.192879,-0.291677,-0.503905,...,0.092566,-0.511123,-0.553887,-0.163939,-1.857091,0.430506,0.012595,0.392483,0.001233,377880
21183,1,-0.037584,0.401981,-0.026370,-0.823541,-0.209837,-0.206982,-0.378039,-0.293631,0.568307,...,1.081964,-0.330496,-0.285179,0.164392,-1.857091,0.430506,0.012595,0.392483,0.012262,2555079
21184,1,-0.125472,0.401981,-0.026370,-0.391795,-0.318444,-0.329195,-0.373842,-0.293016,0.824241,...,1.196125,3.300100,3.030074,-0.039319,-1.857091,0.430506,0.012595,0.392483,0.010695,881214
21185,1,-0.175866,0.401981,-0.026370,-0.998260,-0.181722,-0.132644,-0.100647,-0.293412,0.027134,...,-0.345053,0.446794,0.679211,0.239358,-1.857091,0.430506,0.012595,0.392483,0.006081,9545535


In [105]:
# 피처 각각에 대한 scaling을 수행합니다.
scale_columns = [
    #'HSCD',
 'TRADE_COUNTRYCD',
 'TRADE_HSCD',
 'TARIFF_AVG',
 'SNDIST',
 'NY_GDP_MKTP_CD',
 'NY_GDP_MKTP_CD_1Y',
 'SP_POP_TOTL',
 'PA_NUS_FCRF',
 'IC_BUS_EASE_DFRN_DB',
 'KMDIST',
 #'gdp_diff',
 '1_gdp',
 'item_depend',
 #'country_depend',
 'item_md_rv',
 'sndist_mean',
 'TRADE_HSCD_COUNTRYCD',
 #'FTA',
 #'OECD',
 #'WTO',
 #'ASEM',
 #'ASEAN',
 #'FEALAC',
 '2018_FCL',
 '2018_LCL',
 '2018_EMPTY',
 '2018_Total',
 '2019_FCL',
 '2019_LCL',
 '2019_EMPTY',
 '2019_Total',
 #'18FCL_diff',
 #'18EMPTY_diff',
 #'18Total_diff',
 '2017_hang',
 '2018_hang',
 #'18hang_diff',
 '2018_lpi',
 '2017_fl',
 '2018_fl',
 #'18fl_diff',
 'TARIFF_AVG_sum',
 'TRADE_HSCD_COUNTRYCD_mean',
 'TARIFF_AVG_mean',
 'TRADE_HSCD_COUNTRYCD_sum',  
 #'TRADE_HSCD_COUNTRYCD_ratio'
    
]
picher_df_1 = standard_scaling(test6, scale_columns)
picher_df_1



,HSCD,COUNTRYNM,TRADE_COUNTRYCD,TRADE_HSCD,TARIFF_AVG,SNDIST,NY_GDP_MKTP_CD,NY_GDP_MKTP_CD_1Y,SP_POP_TOTL,PA_NUS_FCRF,...,2018_lpi,2017_fl,2018_fl,18fl_diff,TARIFF_AVG_sum,TRADE_HSCD_COUNTRYCD_sum,TARIFF_AVG_mean,TRADE_HSCD_COUNTRYCD_mean,TRADE_HSCD_COUNTRYCD_ratio,KR_TRADE_HSCD_COUNTRYCD_y
0,1,Algeria,-0.656585,0.114550,2.723872e+00,-1.044902,-0.427436,-0.429401,-0.283572,-0.257200,...,-1.769998,-0.726542,-0.725346,0.140170,1.852499,0.092270,1.798551e+00,0.077059,0.000384,7657
1,1,Australia,-0.276162,0.114550,-3.841857e-01,2.148952,-0.086251,-0.091877,-0.343761,-0.280281,...,0.729715,-0.286765,-0.291912,0.022723,1.852499,0.092270,1.798551e+00,0.077059,0.027318,5294788
2,1,Austria,-0.378754,0.114550,-1.914379e-01,-0.939909,-0.351550,-0.357449,-0.400098,-0.280257,...,1.268114,-0.644223,-0.650505,-0.045383,1.852499,0.092270,1.798551e+00,0.077059,0.028301,335818
3,1,Belgium,0.148010,0.114550,-1.914379e-01,-0.746717,-0.327499,-0.332562,-0.391071,-0.280257,...,1.287343,-0.378461,-0.452831,-0.183586,1.852499,0.092270,1.798551e+00,0.077059,0.043539,173709
4,1,Brazil,-0.381969,0.114550,1.531245e+00,1.397691,0.036551,0.121769,0.300109,-0.280021,...,-0.731656,-0.342032,-0.331365,0.062825,1.852499,0.092270,1.798551e+00,0.077059,0.002311,122850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21184,1,United Arab Emirates,-0.249124,0.085657,-1.871816e-14,-0.395490,-0.360471,-0.366661,-0.397339,-0.279670,...,1.133515,3.000396,2.729067,-0.039319,-1.697112,0.110623,4.127079e-15,0.094147,0.010695,881214
21185,1,Turkey,-0.287730,0.085657,-1.871816e-14,-1.015476,-0.263834,-0.228806,-0.143651,-0.280080,...,-0.423999,0.346164,0.558233,0.239358,-1.697112,0.110623,4.127079e-15,0.094147,0.006081,9545535
21186,1,Egypt,-0.556541,0.085657,-1.871816e-14,-0.916592,-0.407275,-0.410306,-0.087447,-0.278929,...,-0.016561,-0.007706,-0.007690,0.035294,-1.697112,0.110623,4.127079e-15,0.094147,0.001872,1313545
21187,1,United Kingdom,0.651653,0.085657,-1.871816e-14,-0.785173,0.301143,0.297486,-0.199001,-0.280372,...,1.191200,0.596718,0.612247,0.047793,-1.697112,0.110623,4.127079e-15,0.094147,0.067495,19876811


In [106]:
#정답 로그함수로 치환
test["KR_TRADE_HSCD_COUNTRYCD_y"] = np.log1p(test["KR_TRADE_HSCD_COUNTRYCD_y"]) 



In [107]:
#정답 로그함수로 치환
test6["KR_TRADE_HSCD_COUNTRYCD_y"] = np.log1p(test6["KR_TRADE_HSCD_COUNTRYCD_y"]) 




In [108]:
X_train=test.drop([
#'HSCD',
 #'TRADE_COUNTRYCD',
 'TRADE_HSCD',
 'TARIFF_AVG',
 'SNDIST',
 #'NY_GDP_MKTP_CD',
 #'NY_GDP_MKTP_CD_1Y',
 #'SP_POP_TOTL',
 #'PA_NUS_FCRF',
 #'IC_BUS_EASE_DFRN_DB',
 #'KMDIST',
 #'KR_TRADE_HSCD_COUNTRYCD',
 #'gdp_diff',
 #'1_gdp',
 #'item_depend',
 #'country_depend',
 #'item_md_rv',
 #'sndist_mean',
 #'TRADE_HSCD_COUNTRYCD',
 #'FTA',
 #'OECD',
 #'WTO',
 'ASEM',
 #'ASEAN',
 #'FEALAC',
 #'2018_FCL',
 '2018_LCL',
 '2018_EMPTY',
 '2018_Total',
 #'2019_FCL',
 '2019_LCL',
 '2019_EMPTY',
 #'2019_Total',
 '18FCL_diff',
 #'18EMPTY_diff',
 '18Total_diff',
 '2017_hang',
 '2018_hang',
 '18hang_diff',
 #'2018_lpi',
 '2017_fl',
 '2018_fl',
 '18fl_diff',
 #'TARIFF_AVG_sum',
 #'TRADE_HSCD_COUNTRYCD_sum',
 #'TARIFF_AVG_mean',
 #'TRADE_HSCD_COUNTRYCD_mean',
 #'TRADE_HSCD_COUNTRYCD_ratio',
 'KR_TRADE_HSCD_COUNTRYCD_y'
],axis='columns')

X_test =test6.drop([
#'HSCD',
 #'TRADE_COUNTRYCD',
 'TRADE_HSCD',
 'TARIFF_AVG',
 'SNDIST',
 #'NY_GDP_MKTP_CD',
 #'NY_GDP_MKTP_CD_1Y',
 #'SP_POP_TOTL',
 #'PA_NUS_FCRF',
 #'IC_BUS_EASE_DFRN_DB',
 #'KMDIST',
 #'KR_TRADE_HSCD_COUNTRYCD',
 #'gdp_diff',
 #'1_gdp',
 #'item_depend',
 #'country_depend',
 #'item_md_rv',
 #'sndist_mean',
 #'TRADE_HSCD_COUNTRYCD',
 #'FTA',
 #'OECD',
 #'WTO',
 'ASEM',
 #'ASEAN',
 #'FEALAC',
 #'2018_FCL',
 '2018_LCL',
 '2018_EMPTY',
 '2018_Total',
 #'2019_FCL',
 '2019_LCL',
 '2019_EMPTY',
 #'2019_Total',
 '18FCL_diff',
 #'18EMPTY_diff',
 '18Total_diff',
 '2017_hang',
 '2018_hang',
 '18hang_diff',
 #'2018_lpi',
 '2017_fl',
 '2018_fl',
 '18fl_diff',
 #'TARIFF_AVG_sum',
 #'TRADE_HSCD_COUNTRYCD_sum',
 #'TARIFF_AVG_mean',
 #'TRADE_HSCD_COUNTRYCD_mean',
 #'TRADE_HSCD_COUNTRYCD_ratio',
 'KR_TRADE_HSCD_COUNTRYCD_y'
],axis='columns')

y_train = test['KR_TRADE_HSCD_COUNTRYCD_y']

In [109]:
#RF
input_var1 = [    
 'HSCD',
 'TRADE_COUNTRYCD',
 'NY_GDP_MKTP_CD',
 'NY_GDP_MKTP_CD_1Y',
 'SP_POP_TOTL',
 #'PA_NUS_FCRF',
 'IC_BUS_EASE_DFRN_DB',
 'KMDIST',
 'gdp_diff',
 '1_gdp',
 'item_depend',
 'country_depend',
 'item_md_rv',
 'sndist_mean',
 'TRADE_HSCD_COUNTRYCD',
 'FTA',
 'OECD',
 'WTO',
 'ASEAN',
 'FEALAC',
 '2018_FCL',
 '2019_FCL',
 '2019_Total',
 '18EMPTY_diff',
 '2018_lpi',
 'TARIFF_AVG_sum',
 'TRADE_HSCD_COUNTRYCD_sum',
 'TARIFF_AVG_mean',
 'TRADE_HSCD_COUNTRYCD_mean',
 'TRADE_HSCD_COUNTRYCD_ratio']

## 2.2 Submission csv 만들기

In [110]:
sumbmissionf =test6[['KR_TRADE_HSCD_COUNTRYCD_y']]

In [111]:
sumbmissionf =sumbmissionf[0:21190]

In [112]:
sumbmissionf[:]=0

In [113]:
sumbmissionf.to_csv('sumbmissionf_sample.csv',index=False)

In [114]:
sub_f = pd.read_csv('sumbmissionf_sample.csv')

# 3. 2018가장 좋은 성능 모델링으로 2019년 정답 예측

In [115]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
#두 번째 모델 정의
model2 = RandomForestRegressor(random_state=1217, max_features=8,
                              min_samples_leaf=1, n_estimators=500, n_jobs=1)

#모델 학습
model2.fit(X_train[input_var1],y_train)
#예측값 생성
pred2 = model2.predict(X_train[input_var1])
#성능 확인
np.sqrt(mean_squared_error(y_train, pred2))

0.8136124728040751

In [117]:
#X_test 데이터에 대한 예측값 생성
y_pred2 = model2.predict(X_test[input_var1])
sub_f['KR_TRADE_HSCD_COUNTRYCD_y'] = y_pred2

#submission 폴더에 csv파일 젖
sub_f.to_csv('rf_2019_final.csv',index=False)

In [118]:
rf_2019 = pd.read_csv('rf_2019_final.csv')

### 2018년 수입액과 과 2019 예측 수입액 비교

In [121]:
test6[['KR_TRADE_HSCD_COUNTRYCD_y']]

,KR_TRADE_HSCD_COUNTRYCD_y
0,8.943506
1,15.482234
2,12.724328
3,12.065143
4,11.718728
...,...
21184,13.689057
21185,16.071584
21186,14.088241
21187,16.805064


In [119]:
rf_2019

,KR_TRADE_HSCD_COUNTRYCD_y
0,8.480715
1,11.903799
2,11.410803
3,10.683800
4,11.362360
...,...
21184,13.593996
21185,13.013718
21186,11.932625
21187,12.950721


# 4.결론 
- 2019 예측 submission도출

In [122]:
#로그 스케일링 지수함수로 다시 값 변환
rf_2019["KR_TRADE_HSCD_COUNTRYCD_y"] = np.expm1(rf_2019["KR_TRADE_HSCD_COUNTRYCD_y"]) 

In [125]:
rf_2019_exp = rf_2019

In [127]:
#submission 폴더에 csv파일 젖
rf_2019_exp.to_csv('rf_2019_final_exp.csv',index=False)

In [128]:
rf_2019_exp = pd.read_csv('rf_2019_final_exp.csv')

In [129]:
rf_2019_exp

,KR_TRADE_HSCD_COUNTRYCD_y
0,4.819898e+03
1,1.478262e+05
2,9.029087e+04
3,4.364207e+04
4,8.602111e+04
...,...
21184,8.013030e+05
21185,4.485231e+05
21186,1.521494e+05
21187,4.211392e+05
